In [5]:
import numpy as np
from chord_utils import *
from collections import defaultdict

In [6]:
def normalize(r):
    n = [r[0], r[1]]
    while 2 <= n[0] / n[1]:
        n[1] *= 2
    while n[0] / n[1] <= 1:
        n[0] *= 2
    return tuple(n)

# wilson41
# crystal growth algorithm
primes = [3, 5, 7, 11, 13]
ratios = [(1, 1), (2, 1)]
for i, p in enumerate(primes):
    if p == 13:
        break
    r1 = [p, 1]
    max_deg = len(primes) - i
    if p == 3:
        max_deg -= 1
    for j in range(max_deg):
        ratios.append(normalize(tuple(r1)))
        r1i = [r1[1], r1[0]]
        ratios.append(normalize(tuple(r1i)))
        r1[0] *= 3
    if p == 3:
        continue
    r1 = [p, 3]
    for j in range(1, max_deg):
        ratios.append(normalize(tuple(r1)))
        r1i = [r1[1], r1[0]]
        ratios.append(normalize(tuple(r1i)))
        r1[1] *= 3

# diamond tonality
odds = [1, 3, 5, 7, 9, 11]
for p1 in odds:
    for p2 in odds:
        if p1 == p2:
            continue
        if min(p1, p2) != 1 and (max(p1, p2) % min(p1, p2) == 0):
            continue
        r = (p1, p2)
        r = normalize(r)
        if r not in ratios:
            ratios.append(r)

ratios.append((25, 18))
ratios.append((36, 25))
ratios.append((27, 25))
# ratios.append((50, 27))
ratios.append((17, 14))
# ratios.append((28, 17))
ratios.append((21, 17))
# ratios.append((34, 21))
# ratios.sort(key=lambda r: ratio_to_cents(r[0], r[1]))
print('ratios count', len(ratios))
print(ratios)

min_diff = 1200
min_diff_ratios = None
for i, r1 in enumerate(ratios):
    for j, r2 in enumerate(ratios):
        if j <= i:
            continue
        c1 = ratio_to_cents(r1[0], r1[1])
        c2 = ratio_to_cents(r2[0], r2[1])
        diff = min(abs(c1 - c2), abs(1200 - (c1 - c2)))
        if diff < min_diff:
            min_diff = diff
            min_diff_ratios = [r1, r2]
print('min diff ratios', min_diff, min_diff_ratios)


ratios count 53
[(1, 1), (2, 1), (3, 2), (4, 3), (9, 8), (16, 9), (27, 16), (32, 27), (81, 64), (128, 81), (5, 4), (8, 5), (15, 8), (16, 15), (45, 32), (64, 45), (135, 128), (256, 135), (5, 3), (6, 5), (10, 9), (9, 5), (40, 27), (27, 20), (7, 4), (8, 7), (21, 16), (32, 21), (63, 32), (64, 63), (7, 6), (12, 7), (14, 9), (9, 7), (11, 8), (16, 11), (33, 32), (64, 33), (11, 6), (12, 11), (10, 7), (20, 11), (7, 5), (14, 11), (18, 11), (11, 10), (11, 7), (11, 9), (25, 18), (36, 25), (27, 25), (17, 14), (21, 17)]
min diff ratios 7.711522991319498 [(64, 45), (10, 7)]


In [994]:
cents = [1685, 1787, 1868, 2018, 2072, 2254, 2338, 2387, 2570, 2720, 2773, 2885]
print(np.array(cents[1:]) - cents[:-1])
cents_tolerance = 4

for c1 in cents:
    s = str(c1) + ': '
    for c2 in cents:
        add = ''
#         s += str(c2) + '.'
        if c1 == c2:
            add = '.'
        else:
            diff = abs(c1 - c2)
            notes = []
            found = False

            for r in ratios:
                v1 = diff
                v2 = ratio_to_cents(r[0], r[1])
                check = abs(v1 - v2) < cents_tolerance
                if check:
                    if add:
                        add += ','
                    add += str(r[0]) + '/' + str(r[1])
                    found = True
            if not found:
                add += '?'
        add = ' ' * (7 - len(add)) + add
        s += add
    print(s)

[102  81 150  54 182  84  49 183 150  53 112]
1685:       .      ?   10/9  17/14    5/4  25/18      ?    3/2    5/3  20/11   15/8    2/1
1787:       ?      .      ?    8/7      ?  21/16   11/8      ?   11/7   12/7      ?      ?
1868:    10/9      ?      .  12/11    9/8    5/4  21/16  27/20    3/2  18/11  27/16    9/5
2018:   17/14    8/7  12/11      .  33/32      ?      ?  21/17   11/8    3/2      ?      ?
2072:     5/4      ?    9/8  33/32      .   10/9    7/6    6/5    4/3  16/11    3/2    8/5
2254:   25/18  21/16    5/4      ?   10/9      .      ?  27/25    6/5      ?  27/20  36/25
2338:       ?   11/8  21/16      ?    7/6      ?      .      ?    8/7      ?    9/7      ?
2387:     3/2      ?  27/20  21/17    6/5  27/25      ?      .   10/9  17/14    5/4    4/3
2570:     5/3   11/7    3/2   11/8    4/3    6/5    8/7   10/9      .  12/11    9/8    6/5
2720:   20/11   12/7  18/11    3/2  16/11      ?      ?  17/14  12/11      .  33/32  11/10
2773:    15/8      ?  27/16      ?    3/2  2

In [995]:
cents = [1685, 1868, 2072, 2254, 2387, 2570, 2773, 2885]
print(np.array(cents[1:]) - cents[:-1])
cents_tolerance = 1

for c1 in cents:
    s = str(c1) + ': '
    for c2 in cents:
        add = ''
#         s += str(c2) + '.'
        if c1 == c2:
            add = '.'
        else:
            diff = abs(c1 - c2)
            notes = []
            found = False

            for r in ratios:
                v1 = diff
                v2 = ratio_to_cents(r[0], r[1])
                check = abs(v1 - v2) < cents_tolerance
                if check:
                    add += str(r[0]) + '/' + str(r[1])
                    found = True
            if not found:
                add += '?'
        add = ' ' * (6 - len(add)) + add
        s += add
    print(s)

[183 204 182 133 183 203 112]
1685:      .  10/9   5/4 25/18   3/2   5/3  15/8   2/1
1868:   10/9     .   9/8   5/4 27/20   3/2 27/16   9/5
2072:    5/4   9/8     .  10/9   6/5   4/3   3/2   8/5
2254:  25/18   5/4  10/9     . 27/25   6/5 27/20 36/25
2387:    3/2 27/20   6/5 27/25     .  10/9   5/4   4/3
2570:    5/3   3/2   4/3   6/5  10/9     .   9/8   6/5
2773:   15/8 27/16   3/2 27/20   5/4   9/8     . 16/15
2885:    2/1   9/5   8/5 36/25   4/3   6/5 16/15     .


In [2]:
import pyOSC3
import time

In [3]:
client = pyOSC3.OSCClient()
client.connect(('127.0.0.1', 57120))

In [4]:
def send_msg(note, cur_timbre, pan, out_bus_idx, dur):
    try:
        msg = pyOSC3.OSCMessage()
        msg.setAddress("/synths")
        msg.append([note, cur_timbre, pan, out_bus_idx, dur])
        client.send(msg)
    except Exception as e:
        pass

ref_hz = 110
# cents = [1685, 1868, 2072, 2254, 2387, 2570, 2773, 2885]
#             [183   204   182   133   183   203   112]
#
# cents = np.array([2026, 2230, 2412, 2545, 2728])
cents = np.array([2026, 2226, 2409, 2545, 2728])
# cents = np.array([2026, 2230, 2392, 2545, 2728])
# cents = np.array([2068, 2276, 2412, 2545, 2728])
# cents = np.array([1508, 1658, 1787, 1860, 2070])
# cents = np.array([2050, 2259, 2412])
print(cents)
print(np.array(cents[1:]) - cents[:-1])

for c in cents:
    hz = calc_hz(c, ref_hz)
    print(c, hz)
    send_msg(hz, 0, 0, 0.2)
    time.sleep(0.4)


NameError: name 'np' is not defined

In [8]:
tact = 0.35
chords = [
#     [None, 2728, None, 3], # e
#     [None, 2545, None, 2], # lya
#     [None, 2412, None, 2], # lar
#     [None, 2230, None, 2], # de
#     [None, 2545, None, 4], # i
#     [None, 2545, None, 2], # i
#     [2026, 2392, 2728, 4], # la
#     [2047, 2497, 2728, 4], # i
#     [2026, 2480, 2728, 3], # le
#     [2079, 2497, 2750, 1], # i
#     [2044, 2480, 2728, 2], # a
#     [1787, 2339, 2628, 1], # uo # ?
#     [1508, 2412, None, 1], # o # ?
#     [1658, 2076, 2412, 4], # le # ?
#     [1860, 2412, 2545, 2], # si # ?
#     [1860, 2259, 2595, 2], # a # ?
#     [2070] * 3 + [7], # uo # ?

#     [None, 2728, None, 3], # e
#     [None, 2545, None, 2], # lya
#     [None, 2412, None, 2], # lar
#     [None, 2230, None, 2], # de
#     [None, 2545, None, 4], # i
#     [None, 2392, None, 2], # i
    [2026, 2392, 2728, 4], # la
    [2050, 2497, 2728, 4], # i
    [2030, 2460, 2728, 3], # le
    [2090, 2497, 2750, 1], # i
    [2050, 2470, 2728, 2], # a
    [1787, 2339, 2628, 1], # uo # ?
    [1508, 2412, None, 1], # o # ?
    [1658, 2050, 2412, 4], # le # ?
    [1860, 2412, 2545, 2], # si # ?
    [1860, 2259, 2605, 2], # a # ?
    [2050] * 3 + [7], # uo # ?
]

for ch_dur in chords:
    dur = tact * ch_dur[3]
#     ch = ch_dur[:3]
    ch = [ch_dur[0]]
    print(ch)
    prev_c = None
    for c in ch[:3]:
        if prev_c is not None and c is not None and abs(c - prev_c) < 20:
            continue
        prev_c = c
        hz = calc_hz(c, ref_hz)
        send_msg(hz, 0, 0, dur / 3.5)
    time.sleep(dur)


# cents = np.array([2022, 2226, 2408, 2541, 2724])
# print(np.array(cents[1:]) - cents[:-1])
# cents_tolerance = 1

# for c1 in cents:
#     s = str(c1) + ': '
#     for c2 in cents:
#         add = ''
# #         s += str(c2) + '.'
#         if c1 == c2:
#             add = '.'
#         else:
#             diff = abs(c1 - c2)
#             notes = []
#             found = False

#             for r in ratios:
#                 v1 = diff
#                 v2 = ratio_to_cents(r[0], r[1])
#                 check = abs(v1 - v2) < cents_tolerance
#                 if check:
#                     add += str(r[0]) + '/' + str(r[1])
#                     found = True
#             if not found:
#                 add += '?'
#         add = ' ' * (6 - len(add)) + add
#         s += add
#     print(s)

[2026]
[2050]
[2030]
[2090]
[2050]
[1787]
[1508]
[1658]
[1860]
[1860]
[2050]


In [1]:
def d(a, b):
    if a is None or b is None:
        return None
    return a - b

tact = 0.4
tonic = 2026
octave = 1195
edo = 63
step = octave / edo
chords = [
#     [None, 37, None, 4], # e
#     [None, 27, None, 1], # lya
#     [None, 20, None, 1], # lar
#     [None, 11, None, 2], # de
#     [None, 27, None, 4], # i
#     [None, 27, None, 2], # i
#     [0, 19, 37, 4.66], # la # 3/2 (37), -, -
#     [1, 25, 37, 4], # i
#     [0, 24, 37, 3], # le # 3/2 (37), 15/13 (13), 13/10 (24)
#     [3, 25, 38, 1], # i # -, -, 14/11 (22)
#     [1, 24, 37, 2], # a # -, 15/13 (13), 9/7 (23)
#     [-13, 16, 31, 1], # uo # 13/8 (44), 13/11 (15), 11/8 (29)
#     [-28, 19, None, 1], # o
#     [-20, 2, 20, 4], # le # ? 17/11 (40), 17/14 (18), 14/11 (22)
#     [-9, 20, 27, 2], # si # -, 27/25 (7), 11/8 (29)
#     [-9, 13, 30, 2], # a
#     [2] * 3 + [4], # uo # or [1] * 3 + [8]
# #     может это минорная тоника?
    
#     [2, 27, 39, 4], # sa # 3/2 (37), -, -
#     [2, 28, 39, 3], # li # 3/2 (37), 9/8 (11), 4/3 (26)
#     [2, None, 40, 1], # i
#     [-1, 27, 40, 4], # da # 11/7 (41), -, -
#     [2, 20, 40, 2], # ma # -, -, 11/9 (18)
#     [-11, 11, 35, 2], # a # -, -, 14/11 (22)
#     [-17, 28, 45, 3], # ki
#     [-17, 28, 45, 1], # ri # -, -, 18/11 (45)
#     [-20, 18, 39, 2], # a
#     [-22, 9, 33, 1], # uo # 11/6 (55), -, -
#     [None, 16, 10, 1], # o # 16/15 (6)
#     [-20, 2, 18, 4], # le
#     [-9, 18, 36, 2], # si # -, 11/9 (18), -
#     [-9, 11, 28, 2], # a # 3/2 (37), -, -
#     [2] * 3 + [4], # uo
    
#     [0, 26, 38, 4.66], # kra
#     [0, 28, 40, 1.66], # si # -, 8/7 (12), -
#     [-19, 18, 38, 1.66], # uo
#     [-19, 18, 18, 1.66], # o
#     [-10, 9, 28, 3.5], # de
#     [0, 27, 38, 4.66], # kra
#     [-1, 29, 38, 1.66], # si # ? # -, 11/10 (9), -
#     [-18, 19, 19, 3.5], # uo # 3/2 (37), 0, 3/2 (37)
#     [-10, 11, 28, 3.5], # de # correct low note # -, 6/5 (17), -
#     [0, 26, 38, 2], # a # -, 8/7 (12), 4/3 (26)
#     [0, 29, 38, 2], # li # cor top (39) # -, 10/9 (10), 11/8 (29) # how about [0, 29, 38, 2]?
#     [0, 19, 38, 2], # gha
#     [0, 29, 38, 2], # li # cor top (39) # -, 10/9 (10), 11/8 (29) # how about [0, 29, 38, 2]?
#     [0, 27, 38, 3], # me # -, 9/8 (11), -
#     [2, 29, 38, 1], # i
#     [2, 19, 38, 2], # a # correct upper note # -, -, 6/5 (17)
#     [-13, 11, 28, 1], # uo # 11/7 (41), 6/5 (17), -
#     [-27, 11, 28, 1], # o # 11/6 (55), 6/5 (17), -
#     [-19, 2, 19, 4], # le # -, 6/5 (17), -
#     [-8, 20, 28, 2], # si # -, 12/11 (8), -
#     [-8, 12, 28, 2], # a # -, -, 5/4 (20)
#     [2] * 3 + [4], # uo # [3]
    
#     [1, 28, 38, 4], # sa # cor mid (27) # -, 9/8 (11), 4/3 (26)
#     [1, 28, 38, 3], # li # cor low (2) # -, 10/9 (10), -
#     [None, 25, None, 1], # i
#     [1, 28, 38, 4], # da # -, 9/8 (11), -
#     [1, 19, 40, 2], # ma # -, -, 11/9 (18)
#     [-7, 11, 34, 2], # a # 11/7 (41), 9/7 (23), 11/9 (18)
#     [-18, 28, 45, 3], # ki # 2/1 (63), -, -
#     [-16, 28, 45, 1], # ri # -, 6/5 (17), - # how about [-16, 29, 45, 1]?
#     [-18, 18, 38, 2], # a # -, 5/4 (20), -
#     [-21, 11, 32, 1], # uo
#     [-21, 16, 10, 1], # a # 7/5 (31), -, 3/2 (37)
#     [-18, 2, 19, 4], # le # 3/2 (37), 6/5 (17), 5/4 (20)
#     [-8, 20, 39, 2], # si # equalized low notes
#     [-8, 13, 30, 2], # a # equalized low notes # ? -, 6/5 (17), 14/11 (22)
#     [2] * 3 + [8] # uo
]

for ch_dur in chords:
    dur = tact * ch_dur[3]
    ch_steps = ch_dur[:3]
#     ch_steps = [ch_dur[1]]
#     ch_steps = [ch_dur[0], ch_dur[2]]
    ch = []
    for s in ch_steps:
        if s is None:
            ch.append(None)
        else:
            if abs(s) > 100:
                wtf
            ch.append(tonic + s * step)
    if len(ch) == 3:
        diffs = [d(ch[2], ch[0]), d(ch[2], ch[1]), d(ch[1], ch[0])]
        print(diffs, dur)
#     print(ch)
    prev_c = None
    for c in ch[:3]:
        if prev_c is not None and c is not None and abs(c - prev_c) < 20:
            continue
        prev_c = c
        hz = calc_hz(c, ref_hz)
        send_msg(hz, 0, 0, dur / 3.5)
    time.sleep(dur)


[1176.031746031746, 322.4603174603171, 853.5714285714289] 1.2000000000000002


NameError: name 'calc_hz' is not defined

In [10]:
from elia_lrde_chords import *

In [11]:
chords = get_chords()

In [12]:
tact = 0.4
for idx, c in enumerate(chords):
    if idx != len(chords) - 1:
        duration = chords[idx + 1].timestamp - chords[idx].timestamp
#         print(c.syllable, c.timestamp, duration, duration / tact)


In [13]:
# tact = 0.4
# e   : 3*0.4 vs 4
# lya : 1*0.4 vs 1
# lar : 1*0.4 vs 1
# de  : 0.67 (5/3, 1.5) vs 2
# i   : 1.73 (~4) vs 4
# i   : 0.46 (~1) vs 2

# tact = 0.5
# la  : 2.0 (4) vs 4.66
# i   : 1.6 (3) vs 4
# le  : 0.9 (2) vs 3
# i   : 0.52 (1) vs 1
# a   : 0.4 (1) vs 2
# uo  : 0.5 (1) vs 1
# o   : 0.46 (1) vs 1
# le  : 1.5 (3) vs 4
# si  : 0.7 (1.5) vs 2
# a   : 0.7 (1.5) vs 2
# uo  : 2.0 (4) vs 8

# tact = 0.33
# sa  : 1.37 vs 4
# li  : 0.79 vs 3 (2)
# i   : 0.79 vs 1 (2)
# da  : 1.33 vs 4
# ma  : 0.66 vs 2
# a   : 0.84 vs 2 (2.5)
# ki  : 0.91 vs 3
# ri  : 0.36 vs 1
# a   : 0.61 vs 2
# uo  : 0.41 vs 1
# o   : 0.48 vs 1 (1.5)
# le  : 1.63 vs 4 (5)
# si  : 0.63 vs 2
# a   : 0.67 vs 2
# uo  : 2.01 vs 8 (6)

# tact = 0.6 ~ 2*0.33
# kra : 1.8 (3) vs 4.66
# si  : 0.6 (1) vs 1.66
# uo  : 0.53 (1) vs 1.66
# o   : 0.63 (1) vs 1.66
# de  : 1.2 (2) vs 3.5
# tact = 0.5
# kra : 1.53 (3) vs 4.66
# si  : 0.49 (1) vs 1.66
# uo  : 1.14 (2) vs 3.5
# de  : 0.95 (2) vs 3.5
# tact = 0.275
# a   : 0.67 vs 2
# li  : 0.43 vs 2
# gha : 0.48 vs 2
# li  : 0.62 vs 2
# tact = 0.23
# me  : 0.73 vs 3
# i   : 0.23 vs 1
# a   : 0.48 vs 2
# uo  : 0.21 vs 1
# o   : 0.43 vs 1 (2)
# le  : 1.25 vs 4 (5)
# si  : 0.42 vs 2
# a   : 0.62 vs 2 (3)
# uo  : 2.1 vs 8 (10)

# tact = 0.3-0.5
# sa  : 1.4 vs 4
# li  : 1.4 vs 3+1
# i   : 
# da  : 1.6 vs 4
# ma  : 0.6 vs 2
# a   : 0.85 vs 2
# ki  : 1.0 vs 3
# ri  : 0.5 vs 1
# a   : 0.62 vs 2 (1.5)
# uo  : 0.31 vs 1
# o   : 0.48 vs 1
# le  : 1.9 vs 4
# si  : 0.7 vs 2
# a   : 0.8 vs 2
# uo  : ? vs 16

ref_hz = 110
tact = 0.4
tonic = 2026
octave = 1195
edo = 63
step = octave / edo
chords = [
    [None, 37, None, 4], # e
    [None, 27, None, 1], # lya
    [None, 20, None, 1], # lar
    [None, 11, None, 2], # de
    [None, 27, None, 4], # i
    [None, 27, None, 2], # i
    [0, 19, 37, 4], # la # 3/2 (37), -, -
    [1, 25, 37, 4], # i
    [0, 24, 37, 2.5], # le # 3/2 (37), 15/13 (13), 13/10 (24)
    [3, 25, 38, 1*1.5], # i # -, -, 14/11 (22)
    [0, 24, 37, 1*1.33], # a # -, 15/13 (13), 9/7 (23)
    [-13, 16, 31, 1*1.33], # uo # 13/8 (44), 13/11 (15), 11/8 (29)
    [-27, 20, None, 1*1.33], # o
    [-20, 2, 20, 4], # le # ? 17/11 (40), 17/14 (18), 14/11 (22)
    [-9, 20, 27, 2], # si # -, 27/25 (7), 11/8 (29)
    [-9, 13, 30, 2], # a
    [1, 3, 3, 5], # uo # or [1] * 3 + [8]

    [2, 27, 39, 4], # sa # 3/2 (37), -, -
    [2, 28, 39, 2], # li # 3/2 (37), 9/8 (11), 4/3 (26)
    [2, None, 40, 2], # i
    [-1, 27, 40, 4], # da # 11/7 (41), -, -
    [2, 20, 40, 2], # ma # -, -, 11/9 (18)
    [-11, 11, 35, 2], # a # -, -, 14/11 (22)
    [-17, 28, 45, 3], # ki
    [-17, 28, 45, 1], # ri # -, -, 18/11 (45)
    [-20, 18, 39, 2], # a
    [-22, 9, 33, 1], # uo # 11/6 (55), -, -
    [None, 16, 10, 1], # o # 16/15 (6)
    [-20, 2, 18, 4], # le
    [-9, 18, 36, 2], # si # -, 11/9 (18), -
    [-9, 11, 28, 2], # a # 3/2 (37), -, -
    [2] * 3 + [6], # uo
    
    # tact = 0.6 ~ 2*0.33
    # kra : 1.8 (3) vs 4.66
    # si  : 0.6 (1) vs 1.66
    # uo  : 0.53 (1) vs 1.66
    # o   : 0.63 (1) vs 1.66
    # de  : 1.2 (2) vs 3.5
    # tact = 0.5
    # kra : 1.53 (3) vs 4.66
    # si  : 0.49 (1) vs 1.66
    # uo  : 1.14 (2) vs 3.5
    # de  : 0.95 (2) vs 3.5

    [0, 26, 38, 4.5], # kra
    [0, 28, 40, 1.5], # si # -, 8/7 (12), -
    [-19, 18, 38, 1.5], # uo
    [-19, 18, 18, 1.5], # o
    [-10, 9, 28, 3], # de
    [0, 27, 38, 4.5], # kra
    [-1, 29, 38, 1.5], # si # ? # -, 11/10 (9), -
    [-18, 19, 19, 3], # uo # 3/2 (37), 0, 3/2 (37)
    [-10, 11, 28, 3], # de # correct low note # -, 6/5 (17), -
    
    # tact = 0.275
    # a   : 0.67 vs 2
    # li  : 0.43 vs 2
    # gha : 0.48 vs 2
    # li  : 0.62 vs 2
    # tact = 0.23
    # me  : 0.73 vs 3
    # i   : 0.23 vs 1
    # a   : 0.48 vs 2
    # uo  : 0.21 vs 1
    # o   : 0.43 vs 1 (2)
    # le  : 1.25 vs 4 (5)
    # si  : 0.42 vs 2
    # a   : 0.62 vs 2 (3)
    # uo  : 2.1 vs 8 (10)
    [0, 26, 38, 1.5], # a # -, 8/7 (12), 4/3 (26)
    [0, 29, 38, 1.5], # li # cor top (39) # -, 10/9 (10), 11/8 (29) # how about [0, 29, 38, 2]?
    [0, 19, 38, 1.5], # gha
    [0, 29, 38, 1.5], # li # cor top (39) # -, 10/9 (10), 11/8 (29) # how about [0, 29, 38, 2]?
    [0, 27, 38, 2], # me # -, 9/8 (11), -
    [2, 29, 38, 1], # i
    [2, 19, 38, 1.5], # a # correct upper note # -, -, 6/5 (17)
    [-13, 11, 28, 1], # uo # 11/7 (41), 6/5 (17), -
    [-27, 11, 28, 1], # o # 11/6 (55), 6/5 (17), -
    [-19, 2, 19, 4], # le # -, 6/5 (17), -
    [-8, 20, 28, 2], # si # -, 12/11 (8), -
    [-8, 12, 28, 2], # a # -, -, 5/4 (20)
    [2] * 3 + [5], # uo # [3]
    
    
    # tact = 0.3-0.5
    # sa  : 1.4 vs 4
    # li  : 1.4 vs 3+1
    # i   : 
    # da  : 1.6 vs 4
    # ma  : 0.6 vs 2
    # a   : 0.85 vs 2
    # ki  : 1.0 vs 3
    # ri  : 0.5 vs 1
    # a   : 0.62 vs 2 (1.5)
    # uo  : 0.31 vs 1
    # o   : 0.48 vs 1
    # le  : 1.9 vs 4
    # si  : 0.7 vs 2
    # a   : 0.8 vs 2
    # uo  : ? vs 16
    
    [1, 28, 38, 4], # sa # cor mid (27) # -, 9/8 (11), 4/3 (26)
    [1, 28, 38, 3], # li # cor low (2) # -, 10/9 (10), -
    [None, 25, None, 1], # i
    [1, 28, 38, 4], # da # -, 9/8 (11), -
    [1, 19, 40, 2], # ma # -, -, 11/9 (18)
    [-7, 11, 34, 2], # a # 11/7 (41), 9/7 (23), 11/9 (18)
    [-18, 28, 45, 3], # ki # 2/1 (63), -, -
    [-16, 28, 45, 1], # ri # -, 6/5 (17), - # how about [-16, 29, 45, 1]?
    [-18, 18, 38, 2], # a # -, 5/4 (20), -
    [-21, 11, 32, 1], # uo
    [-21, 16, 10, 1], # a # 7/5 (31), -, 3/2 (37)
    [-18, 2, 19, 4], # le # 3/2 (37), 6/5 (17), 5/4 (20)
    [-8, 20, 39, 2], # si # equalized low notes
    [-8, 13, 30, 2], # a # equalized low notes # ? -, 6/5 (17), 14/11 (22)
    [2] * 3 + [8] # uo
]

for ch_dur in chords:
    dur = tact * ch_dur[3]
    ch_steps = ch_dur[:3]
#     ch_steps = [ch_dur[2]]
#     ch_steps = [ch_dur[0], ch_dur[2]]
    ch = []
    for s in ch_steps:
        if s is None:
            ch.append(None)
        else:
            if abs(s) > 100:
                wtf
            ch.append(tonic + s * step)
    if len(ch) == 3:
        diffs = [d(ch[2], ch[0]), d(ch[2], ch[1]), d(ch[1], ch[0])]
        print(diffs, dur)
#     print(ch)
    prev_c = None
    sent = []
    for idx, c in enumerate(ch[:3]):
#         if prev_c is not None and c is not None and abs(c - prev_c) < 20:
#             continue
        prev_c = c
        sent.append(c)
        hz = calc_hz(c, ref_hz)
        send_msg(hz, 0, (idx - 1) / 2, 0, 0.25)#dur / 3.5)
    print(sent) 
    time.sleep(dur)


[None, None, None] 1.6
[None, 2727.8253968253966, None]
[None, None, None] 0.4
[None, 2538.142857142857, None]
[None, None, None] 0.4
[None, 2405.3650793650795, None]
[None, None, None] 0.8
[None, 2234.6507936507937, None]
[None, None, None] 1.6
[None, 2538.142857142857, None]
[None, None, None] 0.8
[None, 2538.142857142857, None]
[701.8253968253966, 341.4285714285711, 360.39682539682553] 1.6
[2026.0, 2386.3968253968255, 2727.8253968253966]
[682.8571428571427, 227.61904761904725, 455.2380952380954] 1.6
[2044.968253968254, 2500.2063492063494, 2727.8253968253966]
[701.8253968253966, 246.58730158730123, 455.2380952380954] 1.0
[2026.0, 2481.2380952380954, 2727.8253968253966]
[663.8888888888891, 246.58730158730168, 417.30158730158746] 0.6000000000000001
[2082.904761904762, 2500.2063492063494, 2746.793650793651]
[701.8253968253966, 246.58730158730123, 455.2380952380954] 0.532
[2026.0, 2481.2380952380954, 2727.8253968253966]
[834.6031746031745, 284.5238095238092, 550.0793650793653] 0.532
[177

KeyboardInterrupt: 

In [7]:
chords = [
    [None, 37, None, 4], # e
    [None, 27, None, 1], # lya
    [None, 20, None, 1], # lar
    [None, 11, None, 2], # de
    [None, 27, None, 4], # i
    [None, 27, None, 2], # i
    [0, 19, 37, 4.66], # la # 3/2 (37), -, -
    [1, 25, 37, 4], # i
    [0, 24, 37, 3], # le # 3/2 (37), 15/13 (13), 13/10 (24)
    [3, 25, 38, 1], # i # -, -, 14/11 (22)
    [1, 24, 37, 2], # a # -, 15/13 (13), 9/7 (23)
    [-13, 16, 31, 1], # uo # 13/8 (44), 13/11 (15), 11/8 (29)
    [-28, 19, None, 1], # o
    [-20, 2, 20, 4], # le # ? 17/11 (40), 17/14 (18), 14/11 (22)
    [-9, 20, 27, 2], # si # -, 27/25 (7), 11/8 (29)
    [-9, 13, 30, 2], # a
    [2] * 3 + [8], # uo # or [1] * 3 + [8]
#     может это минорная тоника?
    
    [2, 27, 39, 4], # sa # 3/2 (37), -, -
    [2, 28, 39, 3], # li # 3/2 (37), 9/8 (11), 4/3 (26)
    [2, None, 40, 1], # i
    [-1, 27, 40, 4], # da # 11/7 (41), -, -
    [2, 20, 40, 2], # ma # -, -, 11/9 (18)
    [-11, 11, 35, 2], # a # -, -, 14/11 (22)
    [-17, 28, 45, 3], # ki # fixme: 62?
    [-17, 28, 45, 1], # ri # -, -, 18/11 (45) # fixme: 62?
    [-20, 18, 39, 2], # a
    [-22, 9, 33, 1], # uo # 11/6 (55), -, -
    [None, 16, 10, 1], # o # 16/15 (6)
    [-20, 2, 18, 4], # le
    [-9, 18, 36, 2], # si # -, 11/9 (18), -
    [-9, 11, 28, 2], # a # 3/2 (37), -, -
    [2] * 3 + [8], # uo
    
    [0, 26, 38, 4.66], # kra
    [0, 28, 40, 1.66], # si # -, 8/7 (12), -
    [-19, 18, 38, 1.66], # uo
    [-19, 18, 18, 1.66], # o
    [-10, 9, 28, 3.5], # de
    [0, 27, 38, 4.66], # kra
    [-1, 29, 38, 1.66], # si # ? # -, 11/10 (9), -
    [-18, 19, 19, 3.5], # uo # 3/2 (37), 0, 3/2 (37)
    [-10, 11, 28, 3.5], # de # correct low note # -, 6/5 (17), -
    [0, 26, 38, 2], # a # -, 8/7 (12), 4/3 (26)
    [0, 29, 38, 2], # li # cor top (39) # -, 10/9 (10), 11/8 (29) # how about [0, 29, 38, 2]?
    [0, 19, 38, 2], # gha
    [0, 29, 38, 2], # li # cor top (39) # -, 10/9 (10), 11/8 (29) # how about [0, 29, 38, 2]?
    [0, 27, 38, 3], # me # -, 9/8 (11), -
    [2, 29, 38, 1], # i
    [2, 19, 38, 2], # a # correct upper note # -, -, 6/5 (17)
    [-13, 11, 28, 1], # uo # 11/7 (41), 6/5 (17), -
    [-27, 11, 28, 1], # o # 11/6 (55), 6/5 (17), -
    [-19, 2, 19, 4], # le # -, 6/5 (17), -
    [-8, 20, 28, 2], # si # -, 12/11 (8), -
    [-8, 12, 28, 2], # a # -, -, 5/4 (20)
    [2] * 3 + [8], # uo # [3]
    
    [1, 28, 38, 4], # sa # cor mid (27) # -, 9/8 (11), 4/3 (26)
    [1, 28, 38, 3], # li # cor low (2) # -, 10/9 (10), -
    [None, 25, None, 1], # i
    [1, 28, 38, 4], # da # -, 9/8 (11), -
    [1, 19, 40, 2], # ma # -, -, 11/9 (18)
    [-7, 11, 34, 2], # a # 11/7 (41), 9/7 (23), 11/9 (18)
    [-18, 28, 45, 3], # ki # 2/1 (63), -, -
    [-16, 28, 45, 1], # ri # -, 6/5 (17), - # how about [-16, 29, 45, 1]?
    [-18, 18, 38, 2], # a # -, 5/4 (20), -
    [-21, 11, 32, 1], # uo
    [-21, 16, 10, 1], # a # 7/5 (31), -, 3/2 (37)
    [-18, 2, 19, 4], # le # 3/2 (37), 6/5 (17), 5/4 (20)
    [-8, 20, 39, 2], # si # equalized low notes
    [-8, 13, 30, 2], # a # equalized low notes # ? -, 6/5 (17), 14/11 (22)
    [2] * 3 + [8] # uo
]

notes = defaultdict(int)
diffs = defaultdict(int)
intervals20 = defaultdict(int)
intervals21 = defaultdict(int)
intervals10 = defaultdict(int)
for c in chords:
    ch = c[:3]
    ch_no_nan = []
    for n in ch:
        if n is not None:
            notes[n] += 1
            ch_no_nan.append(n)
    if len(ch_no_nan) == 3:
        df = (ch_no_nan[2] - ch_no_nan[0], ch_no_nan[2] - ch_no_nan[1], ch_no_nan[1] - ch_no_nan[0])
        diffs[df] += 1
    if d(ch[2], ch[0]) is not None:
        intervals20[d(ch[2], ch[0])] += 1
    if d(ch[2], ch[1]) is not None:
        intervals21[d(ch[2], ch[1])] += 1
    if d(ch[1], ch[0]) is not None:
        intervals10[d(ch[1], ch[0])] += 1
        
keys = sorted(notes.keys())
print([(k, notes[k]) for k in keys])
# print(diffs)
print(intervals20)
print(intervals21)
print(intervals10)

NameError: name 'd' is not defined